In [ ]:
#https://www.dummies.com/programming/python/plotting-a-sound-file-in-ipython/
import matplotlib.pyplot as plt
#from scipy.io import wavfile as wav
from scipy.fftpack import fft
import numpy as np
from gurobipy import *
from IPython.display import Audio
import os
import glob
import untangle

import xml.etree.ElementTree as et
import pandas as pd
import re
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
import soundfile

# Functions

In [ ]:
# Function for transforming a signal with fourier
def fourier_trans(data, rate, begin_time, end_time): 
    # initialising variables
    tp_count = len(data)
    values = np.arange(int(tp_count/2))
    time_period = tp_count/rate
    frequencies = values/time_period
    
    #at what intervals time points are sampled
    sampling_interval = 1/rate;
    
    #time points
    time = np.arange(begin_time, end_time, sampling_interval);
    
    #Frequency domain representation
    fourier_transform = fft(data)/tp_count #normalized the amplitude
    fourier_transform = fourier_transform[range(int(len(data)/2))] #exclude sampling frequency   
    return frequencies, abs(fourier_transform)

# TODO: Funktion überarbeiten damit man es auch versteht!
def new_fft(data, rate):
    N = len(data)

    # sample spacing
    T = 1.0 / rate
    x = np.linspace(0.0, N*T, N)

    yf = fft(data)
    freq = np.linspace(0.0, 1.0/(2.0*T), N//2)
    ampl = 2.0/N * np.abs(yf[0:N//2])
    return freq, ampl

# Frequency domain representation
def plot_freq_domain(freq, fourier, name):
    plt.figure(figsize=(20,5))
    plt.plot(freq, fourier)
    plt.xlabel('Frequency in Hz')
    plt.ylabel('Amplitude')
    plt.xlim(0, 1400)
    plt.title(name)
    plt.show()

# TODO: Warnung herausgeben wenn bei der Teilung der Samplerate eine Fliesskommazahl entsteht
def easy_downsampling(data, rate, sampl_fac):
    ds_data = []
    for i in range(len(data)):
        if (i%sampl_fac == 0):
            ds_data.append(data[i])
    ds_rate = int(rate/sampl_fac)
    return ds_data, ds_rate

#ToDo: startwert von 0 nicht möglich überarbeiten!!
def snip_wav(data, rate, start_sec, end_sec):
    start_point = int((rate * start_sec)-1)
    end_point = int((rate * end_sec)-1)
    #print('start_point:', start_point)
    #print('end_point:', end_point)
    snip_data = data[start_point:end_point]
    #print(snip_data)  
    return snip_data, rate

# Berechnung Vector target
def calc_target_vec(pitch):
    pitch_start = 40
    #pitch_end = 76
    pitch_end = pitch_start + 77 #TODO: muss angepasst werden
    target_vec = []
    
    if pitch_start <= pitch <= pitch_end:
        for pitch_act in range(pitch_start, pitch_end+1):
            
            if pitch_act == pitch:
                target_vec.append(1)
            else:
                target_vec.append(0)
    else:
        print('Pitch muss innerhalb der Range({} - {})!'.format(pitch_start, pitch_end))
    
    return target_vec

def metric(target_v, approx_v):
    norm_factor = np.linalg.norm(approx_v)
    if (norm_factor != 0):
        norm_v = approx_v/norm_factor
        diff = target_v - norm_v
        diff_sum = np.sum(np.abs(diff))
        return diff_sum
    else:
        print('Der Vektor kann nicht normalisiert werden.')
        return approx_v 
    
def norm_vec(vector):
    norm_factor = np.linalg.norm(vector)
    if (norm_factor != 0):
        norm_v = vector/norm_factor
        return norm_v
    else:
        print('Der Vektor kann nicht normalisiert werden.')
        return vector

## Dataframe 

In [ ]:
def read_xml_to_df(path, df_cols, offset_sec, duration_sec, sampl_fac):
    dataset = re.search(r'dataset.*\b', path).group(0)
    path_xml = path + "annotation"
    path_wav = path + "audio"  
    rows = []
    
    for xml_file in sorted(glob.glob(os.path.join(path_xml, '*.xml'))):
        tree = et.parse(xml_file)
        root = tree.getroot()
       
        for globalParam in root.findall('globalParameter'):
            audio_file = globalParam.find('audioFileName').text
            #print(audio_file)
            audio_file = audio_file.replace("\\", "")
            
            # Read wav-file
            wav_file = path_wav + '/' + audio_file
            data, rate = soundfile.read(wav_file)

        for transcription in root.findall('transcription'):

            for event in transcription.findall('event'):
                res = []
                res.append(dataset)
                res.append(audio_file)

                for elem in df_cols[len(res):]:
                    if event is not None:
                        if event.find(elem) is not None:
                            res.append(event.find(elem).text)
                            
                            if elem == 'onsetSec':
                                onsetSec = event.find(elem).text
                                start_sec = round(offset_sec + float(onsetSec), 3)
                                end_sec = round(start_sec + duration_sec, 3)

                                # Snip and downsampling audio-file
                                data_snip, rate_snip = snip_wav(data, rate, start_sec, end_sec)
                                data_down, rate_down = easy_downsampling(data_snip, rate_snip, sampl_fac)
                                
                                # Calc FFT
                                freq, fourier = new_fft(data_down, rate_down)
                                                       
                        elif elem == 'FFT':
                            res.append(fourier)

                        elif elem == 'Freq':
                            res.append(freq)
                        else:
                            res.append(None)
                    else:
                        res.append(None)

                rows.append({df_cols[i]: res[i] for i, _ in enumerate(df_cols)})

    return pd.DataFrame(rows, columns=df_cols)

# Init Dataframe

In [ ]:
# Set audio snippets and sample compression
offset_sec = 0.01
duration_sec = 0.03
sampl_fac = 4

# Create empty Dataframe
df_cols = ['dataset', 'audioFileName', 'pitch', 'onsetSec', 'FFT', 'Freq']
df_ref = pd.DataFrame(columns=df_cols)
df_test = pd.DataFrame(columns=df_cols)

# Create Dataframe with reference tones

In [ ]:
%%time
# paths
path_main = "IDMT-SMT-GUITAR_V2/"
path_main_ref = path_main + "dataset1/Fender Strat Clean Neck SC/"

df_ref = read_xml_to_df(path_main_ref, df_cols, offset_sec, duration_sec, sampl_fac)
df_ref.head()

# Create Dataframe with testdatas

In [ ]:
%%time
# paths testdata
# ToDO: Automatisieren #os.listdir(path_main + path_dataset)

path_testdata = [
    path_main + 'dataset1/Fender Strat Clean Neck SC/',
    path_main + 'dataset1/Fender Strat Clean Neck SC Chords/',
    path_main + 'dataset1/Ibanez Power Strat Clean Bridge HU/',
    path_main + 'dataset1/Ibanez Power Strat Clean Bridge HU Chords/',
    path_main + 'dataset1/Ibanez Power Strat Clean Bridge+Neck SC/',
    path_main + 'dataset1/Ibanez Power Strat Clean Neck HU/',
    path_main + 'dataset2/']

# dataset2:
# [Errno 2] No such file or directory: 'IDMT-SMT-GUITAR_V2/dataset2/audio/\\AR_A_fret_1-20.wav'
# in xml file den audionamen angepasst AR_A_fret_1-20.wav --> AR_A_fret_0-20.wav
# Alle 0-20 und Lick11
# Gesamthaft mussten 27 von 261 XML-Dateien überarbeitet werden

for path in path_testdata[:2]:
    df_act = read_xml_to_df(path, df_cols, offset_sec, duration_sec, sampl_fac)
    df_test = df_test.append(df_act, ignore_index=True)
df_test

In [ ]:
plot_freq_domain(df_ref.Freq[5], df_ref.FFT[5], df_ref.pitch[5])

In [ ]:
plot_freq_domain(df_ref.Freq[6], df_ref.FFT[6], df_ref.pitch[6])

In [ ]:
plot_freq_domain(df_test.Freq[87], df_test.FFT[87], df_test.audioFileName[87])

# Gurobi

In [ ]:
# Build Model
m = Model('AMt')

lambs = []
for lamb in range(len(df_ref)):
    lambs.append(m.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = 'lamb_' + str(lamb)))
m.update()

In [ ]:
'''
%%time
#Optimize Signal

deviation = 0
approxi_sgn = 0
 
for sgn in range(len(mischsignal)):
#for sgn in range(100):
    for lamb in range(len(lambs)):
        approxi_sgn += lambs[lamb] * ref_note[lamb]['fourier'][sgn]
        
    deviation += ((mischsignal[sgn] - approxi_sgn)*(mischsignal[sgn] - approxi_sgn))
    # deviation wird ausmultipliziert stattdessen die für 'mischsignal[sgn] - approxi_sgn'
    # eine Gurobi-Variable (m.addVar  inkl. constr.) verwenden
    # Überprüfen ob der Betrag der Differenz besser geeignet ist?
print('Deviation is calculated.')
'''

In [ ]:
%%time
vec_pred = []

for index, row in df_test.iterrows():
    mischsignal = row.FFT
    #print(row.audioFileName)


    # Schlaufen vertauscht
    cost_function = 0
    approxi_sgn = 0

    for lamb in range(len(lambs)):
        la = lambs[lamb]
        #print('la:', la)
        #note = ref_note[lamb]['fourier']
        note = df_ref.FFT[lamb]

        for sgn in range(len(mischsignal)):
            approxi_sgn = la * note[sgn]
            mi = mischsignal[sgn]
            cost_function += ((mi - approxi_sgn)*(mi - approxi_sgn))
    #print('Cost Function is created')

    m.params.outputflag = 0 # Infotext ausblenden
    m.setObjective(cost_function, GRB.MINIMIZE)
    m.optimize()



    approx_v = []

    for v in m.getVars():
        #print('%s: %g' % (v.varName, v.x))
        approx_v.append(v.x)

    vec_pred.append(approx_v)

In [ ]:
df_test['Vec_Pred'] = vec_pred
df_test.head()

# Calculate Pitch Score

In [ ]:
# Calc all target vectors
target_pitches = df_test.pitch.array
target_vec = []

for pitch in target_pitches:
    target_vec.append(calc_target_vec(int(pitch)))

df_test['Vec_Target'] = target_vec
df_test.head()

In [ ]:
pitch_score = []
vec_target = df_test.pitch.array

for vec in range(len(vec_pred)):
    pitch_score.append(metric(target_vec[vec], vec_pred[vec]))
    
df_test['Pitch_Score'] = pitch_score
df_test.head()

# Calculate F1 Score

In [ ]:
# thresholding vec_pred
%time
vec_pred_thresh = []
v_pred = df_test.Vec_Pred
for ind in range(len(v_pred)):
    vec_thresh = []
    vec = norm_vec(v_pred[ind])
    for i in range(len(vec)):
        if vec[i]> 0.22:
            vec_thresh.append(1)
        else:
            vec_thresh.append(0)
    vec_pred_thresh.append(vec_thresh)
df_test['Thresholded'] = vec_pred_thresh
df_test.head()

In [ ]:
target_vecs = []
for i in df_test.Vec_Target:
    for elem in i:
        target_vecs.append(elem)
#print(target_vecs)

pred_vecs = []
for i in df_test.Thresholded:
    for elem in i:
        pred_vecs.append(elem)
#print(pred_vecs)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(target_vecs,pred_vecs))